In [164]:
import pandas as pd
financial_history = pd.read_csv('financial_results_3.csv')
financial_history['revtq'] = financial_history['revtq'] * 1000000
financial_history['niq'] = financial_history['niq'] * 1000000
financial_history = financial_history[(financial_history['revtq'] >= 15000000)]
financial_history


,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,datacqtr,datafqtr,epspxq,niq,revtq,costat,ggroup,gind,gsector,gsubind
0,1013,19950131,1995,1.0,INDL,C,D,STD,ADCT.1,000886309,...,1994Q4,1995Q1,0.38,10466000.0,121774000.0,I,4520,452010,45,45201020
1,1013,19950430,1995,2.0,INDL,C,D,STD,ADCT.1,000886309,...,1995Q1,1995Q2,0.18,10389000.0,140342000.0,I,4520,452010,45,45201020
2,1013,19950731,1995,3.0,INDL,C,D,STD,ADCT.1,000886309,...,1995Q2,1995Q3,0.26,15674000.0,150454000.0,I,4520,452010,45,45201020
3,1013,19951031,1995,4.0,INDL,C,D,STD,ADCT.1,000886309,...,1995Q3,1995Q4,0.30,18657000.0,173652000.0,I,4520,452010,45,45201020
4,1013,19960131,1996,1.0,INDL,C,D,STD,ADCT.1,000886309,...,1995Q4,1996Q1,0.26,16313000.0,162591000.0,I,4520,452010,45,45201020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194111,347085,20210228,2020,4.0,INDL,C,D,STD,KARO,Y4600W108,...,2021Q1,2020Q4,0.20,4228000.0,42641000.0,A,4510,451030,45,45103010
194112,347085,20210531,2021,1.0,INDL,C,D,STD,KARO,Y4600W108,...,2021Q2,2021Q1,0.25,6417000.0,45522000.0,A,4510,451030,45,45103010
194113,347085,20210831,2021,2.0,INDL,C,D,STD,KARO,Y4600W108,...,2021Q3,2021Q2,0.26,7888000.0,43099000.0,A,4510,451030,45,45103010
194114,347085,20211130,2021,3.0,INDL,C,D,STD,KARO,Y4600W108,...,2021Q4,2021Q3,0.25,7768000.0,36496000.0,A,4510,451030,45,45103010


In [183]:
# financial_history['RevenueGrowthRate'] = financial_history['revtq'].pct_change(periods=4)
# sort values by id and quarter
# shift by negative 4
# check borders (make sure you are heetoi
import numpy as np
from scipy import stats
financial_history['RevenueGrowthRate'] = financial_history.groupby(['cusip'])['revtq'].pct_change(periods=4)
financial_history = financial_history.dropna()
financial_history = financial_history.drop(financial_history[financial_history.RevenueGrowthRate == float('inf')].index)
financial_history = financial_history.drop(financial_history[financial_history.RevenueGrowthRate == float('-inf')].index)
financial_history['NetProfitMargin'] = financial_history['niq']/financial_history['revtq']
financial_history = financial_history.drop(financial_history[financial_history.NetProfitMargin == float('inf')].index)
financial_history = financial_history.dropna()
financial_history['gp_score'] = financial_history['NetProfitMargin'] + financial_history['RevenueGrowthRate']
def create_forward_quarter(row):
    cq = row['datacqtr']
    year = cq[0:4]
    quarter = cq[4:]
    if quarter == 'Q1':
        return year + 'Q2'
    elif quarter == 'Q2':
        return year + 'Q3'
    elif quarter == 'Q3':
        return year + 'Q4'
    elif quarter == 'Q4':
        year = str(int(year) + 1)
        return year + 'Q1'
financial_history['forward_c_qtr'] = financial_history.apply(create_forward_quarter, axis=1)
financial_history

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,revtq,costat,ggroup,gind,gsector,gsubind,RevenueGrowthRate,NetProfitMargin,gp_score,forward_c_qtr
12,1013,19980131,1998,1.0,INDL,C,D,STD,ADCT.1,000886309,...,286396000.0,I,4520,452010,45,45201020,0.115349,0.088720,0.204069,1998Q1
13,1013,19980430,1998,2.0,INDL,C,D,STD,ADCT.1,000886309,...,334635000.0,I,4520,452010,45,45201020,0.198554,0.102613,0.301167,1998Q2
14,1013,19980731,1998,3.0,INDL,C,D,STD,ADCT.1,000886309,...,362496000.0,I,4520,452010,45,45201020,0.235872,0.112021,0.347892,1998Q3
15,1013,19981031,1998,4.0,INDL,C,D,STD,ADCT.1,000886309,...,396151000.0,I,4520,452010,45,45201020,0.181969,0.117059,0.299028,1998Q4
16,1013,19990131,1999,1.0,INDL,C,D,STD,ADCT.1,000886309,...,404294000.0,I,4520,452010,45,45201020,0.411661,-0.023948,0.387713,1999Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194084,339965,20210731,2021,2.0,INDL,C,D,STD,SNOW,833445109,...,272198000.0,A,4510,451020,45,45102030,1.044373,-0.696989,0.347384,2021Q3
194085,339965,20211031,2021,3.0,INDL,C,D,STD,SNOW,833445109,...,334441000.0,A,4510,451020,45,45102030,1.095180,-0.463029,0.632151,2021Q4
194086,339965,20220131,2021,4.0,INDL,C,D,STD,SNOW,833445109,...,383774000.0,A,4510,451020,45,45102030,1.014932,-0.344351,0.670581,2022Q1
194087,339965,20220430,2022,1.0,INDL,C,D,STD,SNOW,833445109,...,422371000.0,A,4510,451020,45,45102030,0.845108,-0.392532,0.452576,2022Q2


In [181]:
market_cap_history = pd.read_csv('market-cap-history.csv')
market_cap_history['Year'] = market_cap_history['MthCalDt'] // 10000
market_cap_history['Month'] = market_cap_history['MthCalDt'].map(str)
market_cap_history['Month'] = market_cap_history['Month'].str[4:8]
market_cap_history['Month'] = market_cap_history['Month'].map(int)
def assign_quarter(row):
    if row['Month'] < 400:
        return 'Q1'
    elif row['Month'] < 700:
        return 'Q2'
    elif row['Month'] < 1000:
        return 'Q3'
    else:
        return 'Q4'
market_cap_history['Quarter'] = market_cap_history.apply(assign_quarter, axis=1)
market_cap_history['Year'] = market_cap_history['Year'].map(str)
market_cap_history['forward_c_qtr'] = market_cap_history['Year'] + market_cap_history['Quarter']
market_cap_history['MthCap'] = market_cap_history['MthCap'].str.replace(',', '')
market_cap_history['MthCap'] = market_cap_history['MthCap'].map(float)
market_cap_history

,HdrCUSIP9,Ticker,ICBIndustry,MthCalDt,MthCap,Year,Month,Quarter,forward_c_qtr
0,233269109,DPAC,NOAVAIL,19950131,32899.50,1995,131,Q1,1995Q1
1,233269109,DPAC,NOAVAIL,19950228,31994.38,1995,228,Q1,1995Q1
2,233269109,DPAC,NOAVAIL,19950331,31537.31,1995,331,Q1,1995Q1
3,233269109,DPAC,NOAVAIL,19950428,29252.00,1995,428,Q2,1995Q2
4,233269109,DPAC,NOAVAIL,19950531,29270.00,1995,531,Q2,1995Q2
...,...,...,...,...,...,...,...,...,...
375531,695255109,PACT,TECH,20131129,535518.72,2013,1129,Q4,2013Q4
375532,695255109,PACT,TECH,20131231,543886.20,2013,1231,Q4,2013Q4
375533,695255109,PACT,TECH,20140131,544646.88,2014,131,Q1,2014Q1
375534,695255109,PACT,TECH,20140228,547689.60,2014,228,Q1,2014Q1


In [184]:
quarterly_avg_market_cap = market_cap_history.groupby(['HdrCUSIP9', 'forward_c_qtr'], as_index=False)['MthCap'].mean()
quarterly_avg_market_cap.rename(columns={'HdrCUSIP9': 'cusip', 'forward_c_qtr': 'forward_c_qtr', 'MthCap': 'QAvgMktCap'}, inplace=True)
quarterly_avg_market_cap



,cusip,forward_c_qtr,QAvgMktCap
0,000794107,1995Q1,115785.833333
1,000794107,1995Q2,111033.916667
2,000794107,1995Q3,135890.000000
3,000794107,1995Q4,170517.520000
4,000794107,1996Q1,233866.480000
...,...,...,...
126802,Y93691106,2010Q3,507375.600000
126803,Y93691106,2010Q4,620360.750000
126804,Y93691106,2011Q1,816955.746667
126805,Y93691106,2011Q2,889203.046667


In [185]:


merged_data = pd.merge(financial_history, quarterly_avg_market_cap, on=['cusip', 'forward_c_qtr'])
merged_data['QAvgMktCap'] = merged_data['QAvgMktCap'] * 1000
merged_data['PS_Ratio'] = merged_data['QAvgMktCap']/merged_data['revtq']
merged_data = merged_data.drop(merged_data[merged_data.PS_Ratio == float('inf')].index)
merged_data = merged_data.drop(merged_data[merged_data.PS_Ratio == float('-inf')].index)
merged_data

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,cusip,...,ggroup,gind,gsector,gsubind,RevenueGrowthRate,NetProfitMargin,gp_score,forward_c_qtr,QAvgMktCap,PS_Ratio
0,1013,19980131,1998,1.0,INDL,C,D,STD,ADCT.1,000886309,...,4520,452010,45,45201020,0.115349,0.088720,0.204069,1998Q1,3.238898e+09,11.309159
1,1013,19980430,1998,2.0,INDL,C,D,STD,ADCT.1,000886309,...,4520,452010,45,45201020,0.198554,0.102613,0.301167,1998Q2,4.238021e+09,12.664607
2,1013,19980731,1998,3.0,INDL,C,D,STD,ADCT.1,000886309,...,4520,452010,45,45201020,0.235872,0.112021,0.347892,1998Q3,3.450648e+09,9.519134
3,1013,19981031,1998,4.0,INDL,C,D,STD,ADCT.1,000886309,...,4520,452010,45,45201020,0.181969,0.117059,0.299028,1998Q4,3.948178e+09,9.966346
4,1013,19990131,1999,1.0,INDL,C,D,STD,ADCT.1,000886309,...,4520,452010,45,45201020,0.411661,-0.023948,0.387713,1999Q1,5.767381e+09,14.265315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59937,315629,20210930,2021,3.0,INDL,C,D,STD,IMOS,16965P202,...,4530,453010,45,45301020,0.276371,0.195199,0.471570,2021Q4,1.496422e+08,0.578770
59938,315629,20211231,2021,4.0,INDL,C,D,STD,IMOS,16965P202,...,4530,453010,45,45301020,0.085938,0.190588,0.276526,2022Q1,1.604747e+08,0.645992
59939,339965,20210731,2021,2.0,INDL,C,D,STD,SNOW,833445109,...,4510,451020,45,45102030,1.044373,-0.696989,0.347384,2021Q3,8.660496e+10,318.168975
59940,339965,20211031,2021,3.0,INDL,C,D,STD,SNOW,833445109,...,4510,451020,45,45102030,1.095180,-0.463029,0.632151,2021Q4,1.041936e+11,311.545450


In [186]:
credit_interest_rates = pd.read_csv('BAMLC0A4CBBBEY.csv')
credit_interest_rates['DATE'] = credit_interest_rates['DATE'].map(str)
credit_interest_rates['DATE'] = credit_interest_rates['DATE'].str.replace('-', '')
credit_interest_rates['DATE'] = credit_interest_rates['DATE'].map(int)
credit_interest_rates['YEAR'] = credit_interest_rates['DATE'] // 10000
credit_interest_rates['Month'] = credit_interest_rates['DATE'].map(str)
credit_interest_rates['Month'] = credit_interest_rates['Month'].str[4:8]
credit_interest_rates['Month'] = credit_interest_rates['Month'].map(int)
def assign_quarter(row):
    if row['Month'] < 400:
        return 'Q1'
    elif row['Month'] < 700:
        return 'Q2'
    elif row['Month'] < 1000:
        return 'Q3'
    else:
        return 'Q4'
credit_interest_rates['Quarter'] = credit_interest_rates.apply(assign_quarter, axis=1)
credit_interest_rates['YEAR'] = credit_interest_rates['YEAR'].map(str)
credit_interest_rates['forward_c_qtr'] = credit_interest_rates['YEAR'] + credit_interest_rates['Quarter']


credit_interest_rates = credit_interest_rates.drop(credit_interest_rates[credit_interest_rates.BAMLC0A4CBBBEY == '.'].index)
credit_interest_rates['bbb_interest_rate'] = credit_interest_rates['BAMLC0A4CBBBEY'].map(float)
quarterly_interest_rates = credit_interest_rates.groupby('forward_c_qtr')['bbb_interest_rate'].mean()
quarterly_interest_rates



forward_c_qtr
1996Q4    7.260000
1997Q1    7.380984
1997Q2    7.513692
1997Q3    7.070462
1997Q4    6.874545
            ...   
2021Q4    2.508636
2022Q1    3.352344
2022Q2    4.661846
2022Q3    5.164030
2022Q4    6.191622
Name: bbb_interest_rate, Length: 105, dtype: float64

In [187]:
full_data = pd.merge(quarterly_interest_rates, merged_data, how='outer', on='forward_c_qtr')
full_data = full_data.dropna()
full_data


,forward_c_qtr,bbb_interest_rate,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,...,costat,ggroup,gind,gsector,gsubind,RevenueGrowthRate,NetProfitMargin,gp_score,QAvgMktCap,PS_Ratio
5,1998Q1,6.724923,1013.0,19980131.0,1998.0,1.0,INDL,C,D,STD,...,I,4520.0,452010.0,45.0,45201020.0,0.115349,0.088720,0.204069,3.238898e+09,11.309159
6,1998Q1,6.724923,1562.0,19980131.0,1997.0,3.0,INDL,C,D,STD,...,A,4510.0,451030.0,45.0,45103010.0,0.230584,0.070422,0.301006,1.504365e+08,5.668295
7,1998Q1,6.724923,1632.0,19980131.0,1998.0,1.0,INDL,C,D,STD,...,A,4530.0,453010.0,45.0,45301020.0,0.088091,0.010003,0.098094,5.145949e+09,16.192873
8,1998Q1,6.724923,1704.0,19980131.0,1998.0,1.0,INDL,C,D,STD,...,A,4530.0,453010.0,45.0,45301010.0,0.564636,0.175037,0.739673,1.281168e+10,9.797221
9,1998Q1,6.724923,1878.0,19980131.0,1997.0,4.0,INDL,C,D,STD,...,A,4510.0,451030.0,45.0,45103010.0,0.574817,0.147822,0.722639,1.956889e+09,10.522778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59942,2022Q1,3.352344,270705.0,20211231.0,2021.0,4.0,INDL,C,D,STD,...,A,4530.0,453010.0,45.0,45301020.0,0.638666,0.315102,0.953768,1.159489e+09,2.565837
59943,2022Q1,3.352344,287252.0,20211231.0,2021.0,3.0,INDL,C,D,STD,...,A,4510.0,451030.0,45.0,45103010.0,0.061840,0.017397,0.079237,8.944729e+09,247.003250
59944,2022Q1,3.352344,298036.0,20211231.0,2021.0,4.0,INDL,C,D,STD,...,A,4510.0,451020.0,45.0,45102020.0,0.024509,0.037450,0.061959,1.882744e+08,1.053313
59945,2022Q1,3.352344,315629.0,20211231.0,2021.0,4.0,INDL,C,D,STD,...,A,4530.0,453010.0,45.0,45301020.0,0.085938,0.190588,0.276526,1.604747e+08,0.645992


In [188]:
ts_group = full_data[(full_data['ggroup'] == 5010)]
ts_group = ts_group[(ts_group['revtq'] >= 15000000)]

high_growth = ts_group[(ts_group['RevenueGrowthRate'] >= 0.30)]
mid_growth = ts_group[(ts_group['RevenueGrowthRate'] < 0.30) & (ts_group['RevenueGrowthRate'] >= 0.20)]
low_growth = ts_group[(ts_group['RevenueGrowthRate'] < 0.20)]

print('Telecommunication Services')
print('+ 30% Revenue Growth')
print(high_growth['PS_Ratio'].describe())
print('+ 20% Revenue Growth')
print(mid_growth['PS_Ratio'].describe())
print('+ < 20% Revenue Growth')
print(low_growth['PS_Ratio'].describe())




Telecommunication Services
+ 30% Revenue Growth
count    810.000000
mean       5.666451
std       13.189237
min        0.001817
25%        0.410068
50%        2.312561
75%        7.333047
max      226.735744
Name: PS_Ratio, dtype: float64
+ 20% Revenue Growth
count    473.000000
mean       4.899102
std       10.887291
min        0.003023
25%        0.529126
50%        1.896056
75%        6.889832
max      185.014640
Name: PS_Ratio, dtype: float64
+ < 20% Revenue Growth
count    4571.000000
mean        4.177416
std         8.004653
min         0.002496
25%         0.590950
50%         2.203363
75%         5.576794
max       252.626889
Name: PS_Ratio, dtype: float64


In [189]:
ss_group = full_data[(full_data['ggroup'] == 4510)]
high_multiple = ss_group[(ss_group['PS_Ratio'] >= 30)]
low_multiple = ss_group[(ss_group['PS_Ratio'] < 30)]
high_growth = ss_group[(ss_group['RevenueGrowthRate'] >= 0.30)]
high_ir_high_growth = high_growth[(high_growth['bbb_interest_rate'] >= 5)]
low_ir_high_growth = high_growth[(high_growth['bbb_interest_rate'] <= 5)]
print(high_multiple['gp_score'].describe())
print(low_multiple['gp_score'].describe())
# print(high_ir_high_growth['PS_Ratio'].describe())
# print(low_ir_high_growth['PS_Ratio'].describe())
# tracking_ps_ratio = high_growth.groupby(['datafqtr'])['PS_Ratio'].mean()
# tracking_ps_ratio.plot(x="datafqtr", y=["PS_Ratio"])





count    1990.000000
mean        0.317204
std         0.440843
min        -2.254534
25%         0.137096
50%         0.304167
75%         0.482724
max         5.795328
Name: gp_score, dtype: float64
count    17835.000000
mean         0.118785
std          0.395526
min         -3.942988
25%         -0.028022
50%          0.141958
75%          0.278592
max          6.416119
Name: gp_score, dtype: float64
